# Imports

In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
export_dir = os.getcwd()
from pathlib import Path
from os import path

from torch.nn import Module
import torch.nn.functional as F

import pickle
import ipynb
import importlib
import shap

from torch.nn import Softmax
softmax = nn.Softmax()

In [ ]:
data_name = "ML1M" ### Can be ML1M, Yahoo, Pinterest
recommender_name = "VAE"
DP_DIR = Path("processed_data", data_name) 
export_dir = Path(os.getcwd())
files_path = Path(export_dir.parent, DP_DIR)
checkpoints_path = Path(export_dir.parent, "checkpoints")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
output_type_dict = {
    "VAE":"multiple",
    "MLP":"single",
    "NCF": "single"}

num_users_dict = {
    "ML1M":6037,
    "Yahoo":13797, 
    "Pinterest":19155}

num_items_dict = {
    "ML1M":3381,
    "Yahoo":4604, 
    "Pinterest":9362}


recommender_path_dict = {
    ("ML1M","VAE"): Path(checkpoints_path, "VAE_ML1M_0.0007_128_10.pt"),
    ("ML1M","MLP"):Path(checkpoints_path, "MLP1_ML1M_0.0076_256_7.pt"),
    ("ML1M","NCF"):Path(checkpoints_path, "NCF_ML1M_5e-05_64_16.pt"),
    
    ("Yahoo","VAE"): Path(checkpoints_path, "VAE_Yahoo_0.0001_128_13.pt"),
    ("Yahoo","MLP"):Path(checkpoints_path, "MLP2_Yahoo_0.0083_128_1.pt"),
    ("Yahoo","NCF"):Path(checkpoints_path, "NCF_Yahoo_0.001_64_21_0.pt"),
    
    ("Pinterest","VAE"): Path(checkpoints_path, "VAE_Pinterest_12_18_0.0001_256.pt"),
    ("Pinterest","MLP"):Path(checkpoints_path, "MLP_Pinterest_0.0062_512_21_0.pt"),
    ("Pinterest","NCF"):Path(checkpoints_path, "NCF2_Pinterest_9e-05_32_9_10.pt"),}


hidden_dim_dict = {
    ("ML1M","VAE"): None,
    ("ML1M","MLP"): 32,
    ("ML1M","NCF"): 8,

    ("Yahoo","VAE"): None,
    ("Yahoo","MLP"):32,
    ("Yahoo","NCF"):8,
    
    ("Pinterest","VAE"): None,
    ("Pinterest","MLP"):512,
    ("Pinterest","NCF"): 64,
}

In [ ]:
from ipynb.fs.defs.help_functions import *
importlib.reload(ipynb.fs.defs.help_functions)
from ipynb.fs.defs.help_functions import *

### Import VAE recommender

In [ ]:
from ipynb.fs.defs.recommenders_architecture import *
importlib.reload(ipynb.fs.defs.recommenders_architecture)
from ipynb.fs.defs.recommenders_architecture import *

# SHAP

## VAE wrapper for shap

In [ ]:
class WrapperModel(nn.Module):
    def __init__(self, model, item_array, cluster_to_items, item_to_cluster, num_items, device, num_clusters=10):
        super(WrapperModel, self).__init__()
        self.model = model
        self.n_items = num_items
        self.cluster_to_items = cluster_to_items
        self.item_to_cluster = item_to_cluster
        self.item_array = item_array
        self.device = device
        self.n_clusters = num_clusters

    def forward(self, input_array):
        batch_size = input_array.shape[0]
        user_vector_batch = torch.zeros(batch_size, self.n_items).to(self.device)

        for cluster in range(input_array.shape[1]):
            cluster_indices = self.cluster_to_items[cluster]
            user_vector_batch[:, cluster_indices] = torch.from_numpy(input_array[:, cluster]).unsqueeze(1).float().to(self.device)

        model_output_batch = self.model(user_vector_batch)
        softmax_output_batch = torch.softmax(model_output_batch, dim=-1)

        user_cluster_scores = []
        for user in range(batch_size):
            user_cluster_scores_per_user = []
            for cluster, items in self.cluster_to_items.items():
                cluster_scores = softmax_output_batch[user, items]
                avg_score = torch.mean(cluster_scores)
                user_cluster_scores_per_user.append(avg_score.unsqueeze(0)) 
            user_cluster_scores.append(torch.cat(user_cluster_scores_per_user)) 

        return torch.stack(user_cluster_scores).cpu().numpy()

### Read data

In [ ]:
train_data = pd.read_csv(Path(files_path,f'train_data_{data_name}.csv'), index_col=0)
test_data = pd.read_csv(Path(files_path,f'test_data_{data_name}.csv'), index_col=0)
train_array = train_data.to_numpy()
test_array = test_data.to_numpy()

## Create / Load top recommended item dict 

In [ ]:
with open(Path(files_path,f'pop_dict_{data_name}.pkl'), 'rb') as f:
    pop_dict = pickle.load(f)
pop_array = np.zeros(len(pop_dict))
for key, value in pop_dict.items():
    pop_array[key] = value
    
output_type = output_type_dict[recommender_name] ### Can be single, multiple
num_users = num_users_dict[data_name] 
num_items = num_items_dict[data_name] 

items_array = np.eye(num_items)
all_items_tensor = torch.Tensor(items_array).to(device)

hidden_dim = hidden_dim_dict[(data_name,recommender_name)]
recommender_path = recommender_path_dict[(data_name,recommender_name)]

kw_dict = {'device':device,
          'num_items': num_items,
           'num_features': num_items, 
            'demographic':False,
          'pop_array':pop_array,
          'all_items_tensor':all_items_tensor,
          'items_array':items_array,
          'output_type':output_type,
          'recommender_name':recommender_name}

In [ ]:
VAE_config= {
"enc_dims": [512,128],
"dropout": 0.5,
"anneal_cap": 0.2,
"total_anneal_steps": 200000}


Pinterest_VAE_config= {
"enc_dims": [256,64],
"dropout": 0.5,
"anneal_cap": 0.2,
"total_anneal_steps": 200000}

def load_recommender():
    if recommender_name=='MLP':
        recommender = MLP(hidden_dim, **kw_dict)
    elif recommender_name=='VAE':
        if data_name == "Pinterest":
            recommender = VAE(Pinterest_VAE_config, **kw_dict)
        else:
            recommender = VAE(VAE_config, **kw_dict)
    elif recommender_name=='NCF':
        MLP_temp = MLP_model(hidden_size=hidden_dim, num_layers=3, **kw_dict)
        GMF_temp = GMF_model(hidden_size=hidden_dim, **kw_dict)
        recommender = NCF(factor_num=hidden_dim, num_layers=3, dropout=0.5, model= 'NeuMF-pre', GMF_model= GMF_temp, MLP_model=MLP_temp, **kw_dict)
    
    recommender_checkpoint = torch.load(Path(checkpoints_path, recommender_path))
    recommender.load_state_dict(recommender_checkpoint)
    recommender.eval()
    for param in recommender.parameters():
        param.requires_grad= False
        
    return recommender


recommender = load_recommender()
optimizer = torch.optim.Adam(recommender.parameters(), lr=0.001)

In [ ]:
create_dicts = True # True if it is the first time running the notebook for the specific recommender and data set
if create_dicts:
    top1_train = {}
    top1_test = {}  
    for i in range(train_array.shape[0]):
        user_index = int(train_data.index[i])
        user_tensor = torch.Tensor(train_array[i]).to(device)
        top1_train[user_index] = int(get_user_recommended_item(user_tensor, recommender, **kw_dict))
    for i in range(test_array.shape[0]):
        user_index = int(test_data.index[i])
        user_tensor = torch.Tensor(test_array[i]).to(device)
        top1_test[user_index] = int(get_user_recommended_item(user_tensor, recommender, **kw_dict))
        
    with open(Path(files_path,f'top1_train_{data_name}_{recommender_name}.pkl'), 'wb') as f:
        pickle.dump(top1_train, f)
    with open(Path(files_path,f'top1_test_{data_name}_{recommender_name}.pkl'), 'wb') as f:
        pickle.dump(top1_test, f)
else:
    with open(Path(files_path,f'top1_train_{data_name}_{recommender_name}.pkl'), 'rb') as f:
        top1_train = pickle.load(f)
    with open(Path(files_path,f'top1_test_{data_name}_{recommender_name}.pkl'), 'rb') as f:
        top1_test = pickle.load(f)

### Clustering

In [ ]:
K = 100
u_train = torch.tensor(train_array).float()
v_train = all_items_tensor
user_ids = np.arange(train_array.shape[0])

In [ ]:
np.random.seed(3)
# Cluster items using k-means
from sklearn.cluster import KMeans
import numpy as np
k = 10

kmeans = KMeans(n_clusters=k)
clusters = kmeans.fit_predict(np.transpose(u_train))

In [ ]:
item_clusters = kmeans.predict(np.transpose(u_train))

# Create mapping from items to clusters
item_to_cluster = {}
# Create mapping from clusters to items
cluster_to_items = {}
for i, cluster in enumerate(item_clusters):
    item_to_cluster[i] = cluster
    if(cluster not in cluster_to_items.keys()):
        cluster_to_items[cluster] = []
    cluster_to_items[cluster].append(i)

In [ ]:
u_test = torch.tensor(test_array).float()

In [ ]:
user_to_clusters = np.zeros((u_test.shape[0],10))

In [ ]:
for i in cluster_to_items.keys():
    user_to_clusters[:,i] = np.sum(u_test.cpu().detach().numpy().T[cluster_to_items[i]], axis=0)

In [ ]:
user_to_clusters_bin =  np.where(user_to_clusters > 0, 1, 0)

In [ ]:
user_to_clusters_train = np.zeros((u_train.shape[0],10))

In [ ]:
user_to_clusters_test = np.zeros((u_test.shape[0],10))

In [ ]:
default_value = 0
target_items_test = list(top1_test.values())
target_items_train = list(top1_train.values())

In [ ]:
for i in cluster_to_items.keys():
    user_to_clusters_train[:,i] = np.sum(u_train.cpu().detach().numpy().T[cluster_to_items[i]], axis=0)

In [ ]:
user_to_clusters_train_bin = np.where(user_to_clusters_train > 0, 1, 0)

In [ ]:
col2 = list(top1_train.values())
input_train_array = np.insert(user_to_clusters_train_bin, 0, col2, axis=1).astype(int)

In [ ]:
for i in cluster_to_items.keys():
    user_to_clusters_test[:,i] = np.sum(u_test.cpu().detach().numpy().T[cluster_to_items[i]], axis=0)

In [ ]:
user_to_clusters_test_bin = np.where(user_to_clusters_test > 0, 1, 0)

In [ ]:
col2 = list(top1_test.values())
input_test_array= np.insert(user_to_clusters_test_bin, 0, col2, axis=1).astype(int)

In [ ]:
wrap_model = WrapperModel(recommender, items_array, cluster_to_items, item_to_cluster, num_items, device)

### SHAP

In [ ]:
K = 50

In [ ]:
sampled_subset = shap.sample(user_to_clusters_train_bin, K)

In [ ]:
explainer = shap.KernelExplainer(wrap_model, sampled_subset)

In [ ]:
shap_values_test = explainer.shap_values(user_to_clusters_bin)

In [ ]:
average_shap = np.mean(shap_values_test, axis=0)

In [ ]:
col1 = np.arange(test_array.shape[0]) + train_array.shape[0]
input_test_array = np.insert(average_shap, 0, col1, axis=1)

In [ ]:
with open(Path(files_path,f'item_to_cluster_{recommender_name}_{data_name}.pkl'), 'wb') as f:
    pickle.dump(item_to_cluster, f)

In [ ]:
with open(Path(files_path,f'shap_values_{recommender_name}_{data_name}.pkl'), 'wb') as f:
    pickle.dump(input_test_array, f)